In [4]:
from pathlib import Path
import numpy as np

lidarlist_path = Path("/root/datasets/semantic-seg/kuangka2/kitti_format")
lidarlist_path

PosixPath('/root/datasets/semantic-seg/kuangka2/kitti_format')

In [5]:
import os

file_list = os.listdir(lidarlist_path)
len(file_list)

6

In [6]:
def read_ascii_pcd(file_path):
    with open(file_path, 'r') as f:
        lines = f.readlines()
    
    points = []
    header = True
    for line in lines:
        if header:
            if line.startswith("DATA"):
                header = False
        else:
            points.append([float(value) for value in line.strip().split()])

    return np.array(points)

In [8]:
for f in file_list:
    file_path = lidarlist_path / f
    # try:
    pcd = read_ascii_pcd(file_path)
    # except ValueError:
    #     print(f"Skipping {file_path}")
    #     continue

    xyz = np.array(pcd[:, :3], dtype=np.float32)
    label = np.array(pcd[:, 3], dtype=np.uint32).reshape(-1, 1)
    
    xyz.tofile((str(file_path))[:-3] + 'bin') 
    label.tofile((str(file_path))[:-3] + 'label')

    # a = np.fromfile((str(file_path))[:-3] + 'bin', dtype=np.float32).reshape(-1, 3)
    # b = np.fromfile((str(file_path))[:-3] + 'label', dtype=np.uint32).reshape(-1, 1)


# pcd.shape, pcd[0], xyz.shape, a.shape, label.shape, b.shape, xyz[0], a[0], label[0], b[0]